In [4]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [5]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec

/home/ubuntu/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

In [7]:
columns = {
    'input_ids': 'uint16',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [8]:
split_by = 1000000

In [9]:
!mkdir partitions-madlad-400-ms

In [ ]:
index = 0
count = 0
a = open(f'partitions-madlad-400-ms/combined-lm-{index}.jsonl', 'w')

with open('prepare-madlad-400-ms.jsonl') as fopen:
    for l in tqdm(fopen):
        a.write(l)
        a.flush()
        count += 1
        if count >= split_by:
            a.close()
            index += 1
            count = 0
            a = open(f'partitions-madlad-400-ms/combined-lm-{index}.jsonl', 'w')
            
a.close()

18992157it [13:23, 8302.52it/s] 

In [12]:
block_size = 4096

def read_dataset(train_file, block_size = block_size):
    
    tokenizer = AutoTokenizer.from_pretrained(
        'malaysia-ai/bpe-tokenizer',
    )
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False
    text_column_name = 'text'
    temp = []
    with open(train_file) as fopen:
        for l in fopen:
            l = msgspec.json.decode(l)
            tokenized = tokenizer(l['text'])['input_ids']
            temp.extend(tokenized)
            while len(temp) >= block_size:
                block = temp[:block_size]
                temp = temp[block_size:]
                if len(block) == block_size:
                    yield np.array(block).astype(np.uint16)

In [13]:
files = sorted(glob('partitions-madlad-400-ms/combined-lm-*.jsonl'), key = lambda x: int(x.split('-')[-1].replace('.jsonl', '')))
files

['partitions-madlad-400-ms/combined-lm-0.jsonl',
 'partitions-madlad-400-ms/combined-lm-1.jsonl',
 'partitions-madlad-400-ms/combined-lm-2.jsonl',
 'partitions-madlad-400-ms/combined-lm-3.jsonl',
 'partitions-madlad-400-ms/combined-lm-4.jsonl',
 'partitions-madlad-400-ms/combined-lm-5.jsonl',
 'partitions-madlad-400-ms/combined-lm-6.jsonl',
 'partitions-madlad-400-ms/combined-lm-7.jsonl',
 'partitions-madlad-400-ms/combined-lm-8.jsonl',
 'partitions-madlad-400-ms/combined-lm-9.jsonl',
 'partitions-madlad-400-ms/combined-lm-10.jsonl',
 'partitions-madlad-400-ms/combined-lm-11.jsonl',
 'partitions-madlad-400-ms/combined-lm-12.jsonl',
 'partitions-madlad-400-ms/combined-lm-13.jsonl',
 'partitions-madlad-400-ms/combined-lm-14.jsonl',
 'partitions-madlad-400-ms/combined-lm-15.jsonl',
 'partitions-madlad-400-ms/combined-lm-16.jsonl',
 'partitions-madlad-400-ms/combined-lm-17.jsonl',
 'partitions-madlad-400-ms/combined-lm-18.jsonl',
 'partitions-madlad-400-ms/combined-lm-19.jsonl',
 'partitio

In [14]:
next(read_dataset(files[1]))

array([    1, 18809, 13691, ...,  2576,   381,   275], dtype=uint16)

In [15]:
!rm -rf tokenized_madlad-400
!mkdir tokenized_madlad-400

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
def loop(files):
    files, index = files
    out_root = f'tokenized_madlad-400/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    with MDSWriter(out=out_root, columns=columns, compression=compression, hashes=hashes) as out:
        for f in files:
            for block in tqdm(read_dataset(train_file = f)):
                sample = {
                    'input_ids': block
                }
                out.write(sample)

In [17]:
import mp
mp.multiprocessing(files, loop, cores = 20, returned = False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [23]:
from streaming import StreamingDataset

total = 0
for f in glob('tokenized_madlad-400/tokenized-*'):
    dataset = StreamingDataset(local = f)
    total += len(dataset)
total * block_size

14983720960